<a href="https://colab.research.google.com/github/Siddharth-Singh-2004/Combined_CU-BEMS_Transformer/blob/main/Combined_CU_BEMS_Transformer_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import math
import copy

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
floor1 = pd.read_csv('/content/drive/MyDrive/2018Floor1.csv')
floor2 = pd.read_csv('/content/drive/MyDrive/2018Floor2.csv')
floor3 = pd.read_csv('/content/drive/MyDrive/2018Floor3.csv')
floor4 = pd.read_csv('/content/drive/MyDrive/2018Floor4.csv')
floor5 = pd.read_csv('/content/drive/MyDrive/2018Floor5.csv')
floor6 = pd.read_csv('/content/drive/MyDrive/2018Floor6.csv')
floor7 = pd.read_csv('/content/drive/MyDrive/2018Floor7.csv')

In [48]:
all_floors = [copy.deepcopy(floor1), copy.deepcopy(floor2), copy.deepcopy(floor3), copy.deepcopy(floor4), copy.deepcopy(floor5), copy.deepcopy(floor6), copy.deepcopy(floor7)]

In [49]:
combined_floors = {}

In [50]:
for num in range(len(all_floors)):

  df_numeric = all_floors[num].drop(columns=['Date'])
  df = df_numeric.groupby(df_numeric.index // 60).sum()
  df_dates = all_floors[num]['Date'].groupby(all_floors[num].index // 60).first()
  df = df_dates.to_frame().join(df)

  df = df.fillna(0)

  grouped_df = df.groupby('Date').sum()
  grouped_df['total_power'] = grouped_df.sum(axis=1)
  df = grouped_df[['total_power']].reset_index()

  df['total_power'] = df['total_power'].astype(float)

  combined_floors[num] = df

In [51]:
combined_floors

{0:                      Date  total_power
 0     2018-07-01 00:00:00      9337.50
 1     2018-07-01 01:00:00      9401.22
 2     2018-07-01 02:00:00      9376.95
 3     2018-07-01 03:00:00      9260.79
 4     2018-07-01 04:00:00      9744.89
 ...                   ...          ...
 4411  2018-12-31 19:00:00      6836.94
 4412  2018-12-31 20:00:00      6725.87
 4413  2018-12-31 21:00:00      6293.01
 4414  2018-12-31 22:00:00      6034.24
 4415  2018-12-31 23:00:00      5508.13
 
 [4416 rows x 2 columns],
 1:                      Date  total_power
 0     2018-07-01 00:00:00     16105.68
 1     2018-07-01 01:00:00     17204.02
 2     2018-07-01 02:00:00     17230.85
 3     2018-07-01 03:00:00     17224.39
 4     2018-07-01 04:00:00     16863.38
 ...                   ...          ...
 4247  2018-12-24 23:00:00       474.42
 4248  2018-12-25 00:00:00       439.50
 4249  2018-12-25 01:00:00       432.57
 4250  2018-12-25 02:00:00       424.50
 4251  2018-12-25 03:00:00        90.51
 
 [42

In [52]:
remove = {}
for floor in range(len(all_floors)):
  remove[floor+1] = []
  for column in all_floors[floor].columns:
    if column != 'Date':
      if 'kW' not in column:
        remove[floor+1].append(column)

In [53]:
remove

{1: [],
 2: ['z1_S1(degC)',
  'z1_S1(RH%)',
  'z1_S1(lux)',
  'z2_S1(degC)',
  'z2_S1(RH%)',
  'z2_S1(lux)',
  'z3_S1(degC)',
  'z3_S1(RH%)',
  'z3_S1(lux)',
  'z4_S1(degC)',
  'z4_S1(RH%)',
  'z4_S1(lux)'],
 3: ['z1_S1(degC)',
  'z1_S1(RH%)',
  'z1_S1(lux)',
  'z2_S1(degC)',
  'z2_S1(RH%)',
  'z2_S1(lux)',
  'z4_S1(degC)',
  'z4_S1(RH%)',
  'z4_S1(lux)',
  'z5_S1(degC)',
  'z5_S1(RH%)',
  'z5_S1(lux)'],
 4: ['z1_S1(degC)',
  'z1_S1(RH%)',
  'z1_S1(lux)',
  'z2_S1(degC)',
  'z2_S1(RH%)',
  'z2_S1(lux)',
  'z4_S1(degC)',
  'z4_S1(RH%)',
  'z4_S1(lux)',
  'z5_S1(degC)',
  'z5_S1(RH%)',
  'z5_S1(lux)'],
 5: ['z1_S1(degC)',
  'z1_S1(RH%)',
  'z1_S1(lux)',
  'z2_S1(degC)',
  'z2_S1(RH%)',
  'z2_S1(lux)',
  'z4_S1(degC)',
  'z4_S1(RH%)',
  'z4_S1(lux)',
  'z5_S1(degC)',
  'z5_S1(RH%)',
  'z5_S1(lux)'],
 6: ['z1_S1(degC)',
  'z1_S1(RH%)',
  'z1_S1(lux)',
  'z2_S1(degC)',
  'z2_S1(RH%)',
  'z2_S1(lux)',
  'z4_S1(degC)',
  'z4_S1(RH%)',
  'z4_S1(lux)',
  'z5_S1(degC)',
  'z5_S1(RH%)',
  'z5_S1(

In [54]:
for floor in range(len(all_floors)):
  all_floors[floor] = all_floors[floor].drop(columns=remove[floor+1])

In [57]:
dataframes = [combined_floors[0], combined_floors[1], combined_floors[2], combined_floors[3], combined_floors[4], combined_floors[5], combined_floors[6]]

# Convert date columns to datetime with error handling
for df in dataframes:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Concatenate all DataFrames
concatenated_df = pd.concat(dataframes)

# Drop rows with invalid dates
concatenated_df = concatenated_df.dropna(subset=['Date'])

# Group by date and sum the total_power columns
result_df = concatenated_df.groupby('Date', as_index=False).sum()

print(result_df)

                    Date  total_power
0    2018-07-01 00:00:00    132184.91
1    2018-07-01 01:00:00    134281.82
2    2018-07-01 02:00:00    133303.13
3    2018-07-01 03:00:00    136726.49
4    2018-07-01 04:00:00    140079.97
...                  ...          ...
4411 2018-12-31 19:00:00      7040.24
4412 2018-12-31 20:00:00      6937.30
4413 2018-12-31 21:00:00      6498.20
4414 2018-12-31 22:00:00      6233.01
4415 2018-12-31 23:00:00      5718.46

[4416 rows x 2 columns]


In [58]:
file_path = '/content/drive/MyDrive/combined.csv'
result_df.to_csv(file_path, index=False)

In [60]:
combined = pd.read_csv('/content/drive/MyDrive/combined.csv')
combined

,Date,total_power
0,2018-07-01 00:00:00,132184.91
1,2018-07-01 01:00:00,134281.82
2,2018-07-01 02:00:00,133303.13
3,2018-07-01 03:00:00,136726.49
4,2018-07-01 04:00:00,140079.97
...,...,...
4411,2018-12-31 19:00:00,7040.24
4412,2018-12-31 20:00:00,6937.30
4413,2018-12-31 21:00:00,6498.20
4414,2018-12-31 22:00:00,6233.01
